In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
raw_data = pd.read_csv("../input/train_data.csv")

print(raw_data.head())

In [ ]:
from sklearn import preprocessing
# Preprocess
# feature_head = ['MidPrice','LastPrice','Volume','BidPrice1','BidVolume1','AskPrice1','AskVolume1']
feature_head = ['MidPrice','LastPrice','BidVolume1', "AskVolume1"]
price_head = ['MidPrice','LastPrice','BidPrice1','AskPrice1']

def get_parse_am_pm(x_date):
    x_df = x_date[1]
    x_df['Hour'] = x_df['Time'].apply(lambda x : int(x[:2]))
    x_am = x_df[x_df['Hour'] <=11]
    x_pm = x_df[x_df['Hour']>11]
    return x_am,x_pm
   
def get_train_data(x_date):
    x_am,x_pm = get_parse_am_pm(x_date)
    return x_am,x_pm

                        
def prepro(x_data):
    x_dataset = []
    y_dataset = []
    for i in range(int(len(x_data)/10)-2):
        standard_price = x_data['MidPrice'].iloc[i*10+9]

        x_data_feature = x_data[feature_head].iloc[i*10:i*10+10]
#         x_data_feature['MidPrice'] = x_data_feature['MidPrice'] - standard_price
        x_data_feature = np.array(x_data_feature).tolist()
#         print(x_data_feature)
        y_item = sum(x_data['MidPrice'].iloc[i*10+10:i*10+30])/20 - standard_price
        
        x_dataset.append(x_data_feature)
        y_dataset.append(y_item)
    return x_dataset, y_dataset


# X_train = pd.DataFrame(columns=feature_head)
# Y_train = pd.DataFrame()
X_train = []
Y_train = []
X_date = raw_data.groupby('Date')
for x_date in X_date:
    # Extract am/pm
    print(x_date[0])
    x_train_am,x_train_pm = get_train_data(x_date)
    x_train_am.sample(frac=1)
    x_train_pm.sample(frac=1)
    
    # Preprocess
    x_data_am, y_data_am = prepro(x_train_am)
    x_data_pm, y_data_pm = prepro(x_train_pm)
    X_train = X_train + x_data_am + x_data_pm
    Y_train = Y_train + y_data_am + y_data_pm

print(len(X_train))
print(len(Y_train))

In [ ]:
def normalize(x_train_set):
    #Normalize
    for i in range(x_train_set.shape[0]):
        minmaxscaler = preprocessing.MinMaxScaler().fit(x_train_set[i,:,3:4])
        x_train_set[i,:,3:4] = minmaxscaler.transform(x_train_set[i,:,3:4])

        minmaxscaler = preprocessing.MinMaxScaler().fit(x_train_set[i,:,2:3])
        x_train_set[i,:,2:3] = minmaxscaler.transform(x_train_set[i,:,2:3])

#     x_train_set[:,:,3] = preprocessing.scale(x_train_set[:,:,3])
#     x_train_set[:,:,2] = preprocessing.scale(x_train_set[:,:,2])
#     x_train_set[:,:,0] = preprocessing.scale(x_train_set[:,0])
    for i in range(x_train_set.shape[0]):
        standard_price = x_train_set[i,-1,0]
        x_train_set[i,:,0] = (x_train_set[i,:,0] - standard_price) * 1000 
        x_train_set[i,:,1] = (x_train_set[i,:,1] - standard_price) * 1000
#     x_train_set[:,:,0] = 
    return x_train_set

x_train_set = np.array(X_train)
y_train_set = np.array(Y_train)
# x_train_set = x_train_set[:,:]
# y_train_set = y_train_set[:,1]
print(x_train_set.shape)
print(y_train_set.shape)

x_train_set = normalize(x_train_set)
# print(x_train_set[0:3])

y_train_set[:] = y_train_set[:]*1000
y_train_set = np.reshape(y_train_set,(y_train_set.shape[0],1))


In [ ]:
from keras.layers import Activation, Dense, LSTM, Dropout
from keras.models import Sequential

activ_func = 'tanh'

neurons = 128
loss = 'mse'                  
optimizer="adam"              
dropout = 0.2      
# batch_size = 12               

# window_len = 7

model = Sequential()
model.add(Dense(neurons, input_shape=(x_train_set.shape[1], x_train_set.shape[2]),))
model.add(LSTM(neurons, return_sequences=True,  activation=activ_func))
model.add(Dropout(dropout))
model.add(LSTM(neurons, return_sequences=False, activation=activ_func))
# model.add(Dropout(dropout))
# model.add(LSTM(neurons, activation=activ_func))
# model.add(Dropout(dropout))
model.add(Dense(24, activation=activ_func))
model.add(Dense(1,activation=activ_func))
model.add(Activation(activ_func))
model.compile(loss=loss, optimizer=optimizer, metrics=['mse'])
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping 

epochs = 20

# early_stopping = EarlyStopping(monitor='loss',patience=2, verbose=0, mode='auto')
history = model.fit(x_train_set,y_train_set,epochs = epochs,batch_size = 32)
# history = model.fit(x_train_set,y_train_set,epochs = epochs,batch_size = 32,callbacks=[early_stopping])

In [ ]:
from sklearn import preprocessing
def normalize(sss):
    #Normalize
    
    minmaxscaler = preprocessing.MinMaxScaler().fit(sss[0,:,3:4])
    sss[0,:,3:4] = minmaxscaler.transform(sss[0,:,3:4])
    
    minmaxscaler = preprocessing.MinMaxScaler().fit(sss[0,:,2:3])
    sss[0,:,2:3] = minmaxscaler.transform(sss[0,:,2:3])
    
    for i in range(sss.shape[0]):
        standard_price = sss[i,-1,0]
        sss[i,:,0] = (sss[i,:,0] - standard_price) * 1000
        sss[i,:,1] = (sss[i,:,1] - standard_price) * 1000
#         minmaxscaler = preprocessing.MinMaxScaler().fit(x_train_set[i:i+1,:,0])
#         x_train_set[i:i+1,:,0] = minmaxscaler.transform(x_train_set[i:i+1,:,0])
#     x_train_set[:,:,0] = 
    return sss

test_data = pd.read_csv("../input/test_data.csv")
print(len(test_data))
# print(test_data[feature_head].loc[0:10])
ans_list = []
ans_for_hist = []
ans_csv = pd.DataFrame(columns=['caseid','midprice'])

for i in range(142,1000):
    standard_price = test_data['MidPrice'].iloc[i*10+9]

    X_test = test_data[feature_head].iloc[i*10:i*10+10]
    X_test = np.array(X_test)
    X_test = X_test.reshape(1,X_test.shape[0],X_test.shape[1])
    
    X_test = normalize(X_test)
#     print(X_test)
    #     X_test = np.array(X_test)
#     print(X_test.shape)
    Y_test = model.predict(X_test)
#     print(Y_test)
    ans = standard_price + Y_test[0,0]/1000
    
    ans_for_hist = ans_for_hist + [Y_test[0,0]/1000]
#     ans = standard_price + sum(Y_test[:,0,0])/len(Y_test[:,0,0])
    
    ans_list = ans_list + [ ans ]
    ans_csv.loc[i-142]=[i+1,ans]
# plt.hist(ans)
plt.hist(ans_for_hist)
# print(ans)
ans_csv.to_csv('data.csv', index = False, index_label = False)

In [ ]:
ans_list